In [1]:
import numpy as np

import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)

2.0.0


In [2]:
# !pip install tensorflow==2.0.0
# import tensorflow as tf
# print(tf.__version__)

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json \
    -O /tmp/sarcasm.json

--2020-08-22 23:25:30--  https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2020-08-22 23:25:30 (186 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [4]:
# remove stop words
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [5]:

with open("/tmp/sarcasm.json", 'r') as f:
  datastore = json.load(f)

plain_sentences = []
sentences = []
labels = []
urls = []
for item in datastore:
  labels.append(item['is_sarcastic'])
  sentence = item['headline'].lower()
  plain_sentences.append(sentence)
  for word in stopwords:
    token = " " + word + " "
    sentence = sentence.replace(token, " ")
  sentences.append(sentence)

print(len(labels))
print(len(sentences))
print(sentences[1])
print(plain_sentences[1])
print(labels[1])


26709
26709
the 'roseanne' revival catches thorny political mood, better worse
the 'roseanne' revival catches up to our thorny political mood, for better and worse
0


In [6]:
# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [7]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [8]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

In [9]:
word_index = tokenizer.word_index
print(len(word_index))

29642


In [10]:
print(tf.__version__)

2.0.0


In [11]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [12]:
# need to convert them to np array

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [15]:
model = tf.keras.Sequential([
    # tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.MaxPooling1D(pool_size=4),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # tf.keras.layers.Dense(24, activation='relu'),
    # tf.keras.layers.Dense(1, activation='sigmoid')

    # # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    # # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # # tf.keras.layers.Dense(24, activation='relu'),
    # # tf.keras.layers.Dense(1, activation='sigmoid')
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 24)                1560      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 30,129
Trainable params: 30,129
Non-trainable params: 0
_________________________________________________________________


In [20]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Train on 20000 samples, validate on 6709 samples
Epoch 1/50
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.1502 - accuracy: 0.9304 - val_loss: 1.2324 - val_accuracy: 0.7563
Epoch 2/50
20000/20000 [==============================] - 42s 2ms/sample - loss: 0.1501 - accuracy: 0.9313 - val_loss: 1.3721 - val_accuracy: 0.7554
Epoch 3/50
20000/20000 [==============================] - 42s 2ms/sample - loss: 0.1430 - accuracy: 0.9329 - val_loss: 1.3881 - val_accuracy: 0.7602
Epoch 4/50
20000/20000 [==============================] - 43s 2ms/sample - loss: 0.1411 - accuracy: 0.9341 - val_loss: 1.4844 - val_accuracy: 0.7582
Epoch 5/50
20000/20000 [==============================] - 43s 2ms/sample - loss: 0.1349 - accuracy: 0.9359 - val_loss: 1.5973 - val_accuracy: 0.7566
Epoch 6/50
20000/20000 [==============================] - 43s 2ms/sample - loss: 0.1302 - accuracy: 0.9381 - val_loss: 1.5785 - val_accuracy: 0.7550
Epoch 7/50
20000/20000 [==============================] -

In [21]:
model.save('1bi_2_updated.h5')